In [ ]:
# Install necessary Python pakcages
# rdflib: RDF composition and querying with SPARQL in Python.
from rdflib import RDFS, RDF, Namespace, Graph, URIRef

# Other imports
from common import *

## 1. Initialize a graph and define namespaces
1. Initialize a graph in rdflib. A graph in RDF is like a whiteboard where you will put your triples.
2. A URI represents an entity and a URI consists of a namespace and an identifier. All URIs should be unique and namespaces are prefixes that help avoiding reusing the same identifiers in other domains.
  - Namespace: "http://example.com#", Identifier: "AHU-1", -> URI: "http://example.com#AHU-1"
  - E.g., If you want to use AHU-1 for Building-A and Building-B, you may want to use different namespaces for different buildings while using AHU-1 consistently.
3. We use following namespaces:
  - RDFS and RDF are namespaces defined for general purpose in Semantic Web.
  - BRICK is Brick's namespace for TagSets. 
  - EX is our building's namespace. We will intantiate our entities under this namespace.

In [ ]:
g = Graph() # Initialize a graph
RDFS # predefined namespace as 'http://www.w3.org/2000/01/rdf-schema#'
RDF # predefined namespace as 'http://www.w3.org/1999/02/22-rdf-syntax-ns#'
BRICK = Namespace('https://brickschema.org/schema/1.0.1/Brick#')
BF = Namespace('https://brickschema.org/schema/1.0.1/BrickFrame#')
EX = Namespace('http://example.com#')
g.bind('ex', EX)
g.bind('brick', BRICK)
g.bind('bf', BF)
g.bind('rdfs', RDFS)
g.bind('rdf', RDF)

## 2. Adding triples
### 2.1. Your fisrt triple.
Each triple consists of Subject-Predicate-Object. Each of S-P-O is a URI. Let's make "*AHU-1 is an instance of AHU TagSet*". AHU-1 is the name of our instance in EX. AHU in Brick is a TagSet. AHU-1 and AHU is associated with instantiation relationship. 

- In RDF (Turtle syntax), their full URIs are:
```turtle
<http://example.com#AHU-1>
<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>
<https://brickschema.org/schema/1.0.1/Brick#AHU>
```
- It can be reduced by using namespace prefixes (Turtle syntax)
```turtle
ex:AHU-1
rdf:type # It is often written as "a", which is an abbreviation of "is a".
brick:AHU
```
- Python RDFLib
```python
EX['AHU-1']
RDF['type']
BRICK['AHU']
```

Triple is the minimum unit that you can add into a graph. In is represented as just enumeration of S-P-O in a line in Turtle and a tuple in RDFLib. Below examples represent same things.

- Turtle with full namespaces
```turtle
<http://example.com#AHU-1> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://brickschema.org/schema/1.0.1/Brick#AHU> .
```
- Turtle with namesapce abbreviations
```turtle
ex:AHU-1 rdf:type brick:AHU .
```
- RDFLib (Python code)
```python
(EX['AHU-1'], RDF['type'], BRICK['AHU'])
```

We will use RDFLib for composition in Python and serialize them in Turtle.

In [ ]:
# Add the triple using RDFLib's syntactic sugar.

first_triple1 = (EX['AHU-1'], RDF.type, BRICK['AHU']) # AHU-1 is an instance of a TagSet, BLD-A
print('The triple in Python:')
print(first_triple1)
print('\nThe triple in Turtle:')
g.add(first_triple1)
print_graph(g)
# "a" is a common abbreviation of "rdf:type"'

In [ ]:
# Add the triple using RDFLib with bare URIs.
# first_triple1 is same as first_triple2 but in different style.

first_triple2 = (URIRef('http://example.com#AHU-1'), URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://brickschema.org/schema/1.0.1/Brick#AHU'))
g.add(first_triple2)
print_graph(g)

<img src="files/composition/ahu.png">

### 2.2.  Add location information for AHU-1
1. Let's add other information, AHU-1's location. Say AHU-1 is located in Room-B100, which is in the basement.
  - Say the basement's URI is [ex:Room-B100](http://example.com#Room-B2150).
  - Define the basement is an instance of [brick:Basement](https://brickschema.org/schema/1.0.1/Brick#Basement).
  - AHU-1 is located in BLD-A. [bf:isLocatedIn](https://brickschema.org/schema/1.0.1/BrickFrame#isLocatedIn).

In [ ]:
# Use RDFLib syntax hereafter

# Rm-B100 is an instance of Basement.
g.add((EX['Room-B100'], RDF['type'], BRICK['Basement'])) 

# AHU-1 is located in Room-B100.
g.add((EX['AHU-1'], BF['isLocatedIn'], EX['Room-B100']))

print_graph(g)

<img src="files/composition/ahu_room.png">

### Excercise 2.1. Add other information for AHU-1
1. We stated that "AHU-1" is an instance of Brick's AHU TagSet. What kind of information do you want to put more? Let's add those:  
  - Define VAV-2150 is an instance of VAV (https://brickschema.org/schema/1.0.1/Brick#VAV)
  - Define AHU-1 feeds air to VAV-2150 (https://brickschema.org/schema/1.0.1/BrickFrame#feeds)

In [ ]:
# VAV-2150 is an instance of VAV
g.add((EX['VAV-2150'], RDF['type'], BRICK['VAV']))

# AHU-1 feeds VAV-2150
g.add((EX['AHU-1'], BF['feeds'], BRICK['VAV-2150']))

print_graph(g)

### Expected Output:
```turtle
@prefix bf: <https://brickschema.org/schema/1.0.1/BrickFrame#> .
@prefix brick: <https://brickschema.org/schema/1.0.1/Brick#> .
@prefix ex: <http://example.com#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:AHU-1 a brick:AHU ;
    bf:feeds ex:VAV-2150 ;
    bf:isLocatedIn ex:Room-B100 .

ex:Room-B100 a brick:Basement .

ex:VAV-2150 a brick:VAV .
```

<img src="files/composition/ahu_room_vav.png">

## Excercise 2.2. Add a Zone associated with VAV-2150
1. Let's define Zone that VAV-2150 is feeding.
  - Define Zone-2150 is an instance of HVAC_Zone
  - Define VAV-2150 feeds Zone-2150
  - Define AHU-1 feeds VAV-2150

In [ ]:
# Zone-2150 is an instance of HVAC_Zone
g.add((EX['Zone-2150'], RDF['type'], BRICK['HVAC_Zone']))

# VAV-2150 feeds Zone-2150
g.add((EX['VAV-2150'], BF['feeds'], EX['Zone-2150']))

print_graph(g)

### Expected Output:
```turtle
@prefix bf: <https://brickschema.org/schema/1.0.1/BrickFrame#> .
@prefix brick: <https://brickschema.org/schema/1.0.1/Brick#> .
@prefix ex: <http://example.com#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:AHU-1 a brick:AHU ;
    bf:feeds brick:VAV-2150 ;
    bf:isLocatedIn ex:Room-B100 .

ex:VAV-2150 a brick:VAV ;
    bf:feeds ex:Zone-2150 .

ex:Room-B100 a brick:Basement .

ex:Zone-2150 a brick:HVAC_Zone .
```

<img src="files/composition/ahu_room_vav_zone.png">

### Excercise 2.3. Add a Zone_Temperature_Sensor called ZNT-2150 and associate it with others.
1. Define ZNT-2150 as an instance of Zone_Temperature_Sensor
2. ZNT-2150 is point of VAV-2150 (i.e., ZNT-2150 functions for VAV-2150)
3. ZNT-2150 is located in Zone-2150.

In [ ]:
# ZNT-2150 is an instance of Zone_Temperature_Sensor
g.add((EX['ZNT-2150'], RDF['type'], BRICK['Zone_Temperature_Sensor']))

# ZNT-2150 is point of VAV-2150
g.add((EX['ZNT-2150'], BF['isPointOf'], EX['VAV-2150']))

# ZNT-2150 is located In Zone-2150
g.add((EX['ZNT-2150'], BF['isLocatedIn'], EX['Zone-2150']))

print_graph(g)

### Expected Output:
```turtle
ex:AHU-1 a brick:AHU ;
    bf:feeds brick:VAV-2150 ;
    bf:isLocatedIn ex:Room-B100 .

ex:ZNT-2150 a brick:Zone_Temperature_Sensor ;
    bf:isLocatedIn ex:Zone-2150 ;
    bf:isPointOf ex:VAV-2150 .

ex:Room-B100 a brick:Basement .

ex:VAV-2150 a brick:VAV ;
    bf:feeds ex:Zone-2150 .

ex:Zone-2150 a brick:HVAC_Zone .
```

<img src="files/composition/ahu_room_vav_zone_znt.png">

## 3. Save/Load the graph in Turtle 
1. We can save and load the graph. We stick to Turtle format among multiple possible serialization formats.
2. When you load, you can load multiple graph files into a graph.

In [ ]:
# Save the graph
g.serialize(destination='sample_building_sol.ttl', format='turtle') # You can open the file in jupyter GUI.

# Load the graph
g = Graph() # Initialize a new graph.
g.parse('sample_building_sol.ttl', format='turtle') # Load the stored graph.
g.parse('Brick/dist/Brick.ttl', format='turtle') # Load Brick schema.

## 4. SPARQL Basic
1. [SPARQL](https://www.w3.org/TR/sparql11-query/) is a standard query langauge over RDF.
2. You can define variables and a query pattern, then the SPARQL querying engine returns instances matching the pattern.
3. Define variables followed by a question mark (?) after "select". Define a desired pattern inside "where". 
  - Find ?s where any ?p and any ?o are associated with ?s as predicate and object individually. 
      ```SPARQL
      select ?s where {
      ?s ?p ?o .
      }
      ```
    This returns all subjects in the entire graph.
   - Find ?s where ?s is a type of any subclasses of Sensor and also ?s is located in Room-101.
       ```SPARQL
       PREFIX brick: <https://brickschema.org/schema/1.0.1/Brick#>
       PREFIX bf: <https://brickschema.org/schema/1.0.1/BrickFrame#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
       PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
       PREFIX ex: <http://example.com#>
       select ?s where {
         ?sensortype rdfs:subClassOf+ brick:Sensor .
         ?s rdf:type ?sensortype .
         ?s bf:isLocatedIn ex:Room-101 .
       }
       ```

### 4.1. Iterating SPARQL Results in RDFLib.

- A row of results in SPARQL is a tuple of variables in the query.
- Let's see the result of the very basic query.

In [ ]:
### Querying All S, P and O.

g = Graph()
g.parse('sample_building_sol.ttl', format='turtle')
q = """
select ?s ?p ?o where {
?s ?p ?o .
}
"""
res = g.query(q)
for row in res:
    print(row)

In [ ]:
### Querying Only S and P 
# ?o is inside the where clauses but not in the varible declarations. 
# ?o will be considered for the pattern but not included in the answer..

q = """
select ?s ?p where {
?s ?p ?o .
}
"""
res = g.query(q)
for row in res:
    print(row)

### 4.2. Subclass and intsance relationships

In [ ]:
# Load the graph with Brick for querying.
g = Graph() # Initialize a new graph.
g.parse('sample_building_sol.ttl', format='turtle') # Load the stored graph.
g.parse('Brick/dist/Brick.ttl', format='turtle') # Load Brick schema. We need it to exploit the hierarchy.

In [ ]:
### Find ?s where ?s is an instance of Zone_Temperature_Sensor.

q = """
PREFIX brick: <https://brickschema.org/schema/1.0.1/Brick#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
select ?s where {
?s rdf:type brick:Zone_Temperature_Sensor.
}
"""

res = g.query(q)
for row in res:
    print(row)

In [ ]:
### Find ?s where ?s is any type of Sensor.

q = """
PREFIX brick: <https://brickschema.org/schema/1.0.1/Brick#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
select ?s where {
?s rdf:type ?sensorClass .
?sensorClass rdfs:subClassOf+ brick:Sensor .
}
"""
res = g.query(q)
for row in res:
    print(row)

## Excercise 4.1. Find zones.
1. Let's find zones. Use the rdf:type and/or rdfs:subClassOf+ 

In [ ]:
q = """
PREFIX brick: <https://brickschema.org/schema/1.0.1/Brick#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bf: <https://brickschema.org/schema/1.0.1/BrickFrame#>

select ?z where {
?z rdf:type brick:HVAC_Zone .
#?z rdf:type/rdfs:subClassOf brick:Zone . # use this if you want to be more agnostic to zone types.
}
"""
res = g.query(q)
for row in res:
    print(row)

### Expected Output:
(rdflib.term.URIRef('http://example.com#Zone-2150'),)


## Excercise 4.2. Find zones fed+ by AHU-1.
1. Extend Excercise 3.1. to relate feeds relationships with the zones found there.

In [ ]:
# VAV-2150 is an instance of VAV
q = """
PREFIX brick: <https://brickschema.org/schema/1.0.1/Brick#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bf: <https://brickschema.org/schema/1.0.1/BrickFrame#>
select ?z where {
?z rdf:type brick:HVAC_Zone .
ex:AHU-1 bf:feeds+ ?z .
}
"""
res = g.query(q)
for row in res:
    print(row)

### 4.3. Querying tuples.
1. SPARQL is capable of finding tuples of variables. The variables after select constitutte a tuple format.

In [ ]:
q = """
PREFIX brick: <https://brickschema.org/schema/1.0.1/Brick#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX bf: <https://brickschema.org/schema/1.0.1/BrickFrame#>
select ?s ?r where {
  ?s bf:isLocatedIn ?r .
  ?s rdf:type brick:Zone_Temperature_Sensor .
  ?r rdf:type brick:HVAC_Zone .
}

"""
res = g.query(q)
for row in res:
    print(row)